In [ ]:
from RADARDataCapture import FMCWRADARDataCapture
from MOCAPDataCapture import MOCAPDataCapture
from FPDataCapture import FPDataCapture
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import statistics
from scipy.stats import pearsonr

Load Times and Frames Table

In [ ]:
event_csv_path = "/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/MOCAP_FP_RADAR_FU_Stable_Break_FD_TIME_FRAMES_v3.csv"

event_labels_df = pd.read_csv(event_csv_path)

fp_mocap_comparison_df = event_labels_df
fp_mocap_comparison_df.drop(columns=['t_foot_up', 'MOCAP_Start_Time', 'RADAR_Start_Frame', 'RADAR_End_Frame', 'MOCAP_End_Time'], inplace=True)

fp_mocap_comparison_df['frame_break'] = fp_mocap_comparison_df['frame_break'].fillna(fp_mocap_comparison_df['frame_end'])

fp_mocap_comparison_df['t_break'] = fp_mocap_comparison_df['t_break'].fillna(fp_mocap_comparison_df['t_foot_down'])

fp_mocap_comparison_df['frame_break'] = fp_mocap_comparison_df['frame_break'].astype(int)
print(fp_mocap_comparison_df.head())

Avg COP Time-Normalized Distance by Frame

In [ ]:
# main.py

import matplotlib.pyplot as plt

# Import the FPDataCapture class from its module
# from your_module import FPDataCapture  # Uncomment and modify this line according to your file structure

# Set the base file path to your data file
participant = "04"
MOCAP_FP_capture_name = "04_MNTRL_MC_V1"
base_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_FP_1and2_MNTR/{participant}/{MOCAP_FP_capture_name}.tsv"
# Instantiate the FPDataCapture object
force_plate_capture = FPDataCapture(base_file_path)

# Calculate average COP velocity per frame
force_plate_capture.calculate_avg_COP_velocity_per_frame()

# Check if the dictionary is populated
if not force_plate_capture.avg_COP_per_frame_dict:
    print(f"No average COP velocity data to plot for capture {force_plate_capture.RADAR_Capture}.")
else:
    # Plot the average COP velocity per frame for each tx
    for tx, avg_velocities in force_plate_capture.avg_COP_per_frame_dict.items():
        if not avg_velocities:
            print(f"No data to plot for tx {tx} in capture {force_plate_capture.RADAR_Capture}.")
        plt.figure(figsize=(12, 6))
        plt.plot(avg_velocities, marker='o', linestyle='-')
        plt.xlabel('Frame Index')
        plt.ylabel('Average COP Velocity (mm/s)')
        plt.title(f'Average COP Velocity per Frame for {force_plate_capture.RADAR_Capture}, tx {tx}')
        plt.grid(True)
        plt.show()
        break

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Import the FPDataCapture class from its module
# from your_module import FPDataCapture  # Uncomment and modify this line according to your file structure
# Iterate over each row in the DataFrame
for index, row in fp_mocap_comparison_df.iterrows():
    participant = row['RADAR_capture'][:2]
    MOCAP_FP_capture_name = row['RADAR_capture'].replace('_RR_', '_MC_')
    base_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_FP_1and2_MNTR/{participant}/{MOCAP_FP_capture_name}.tsv"
    
    # Instantiate the FPDataCapture object with is_foot_always_up=True
    force_plate_capture = FPDataCapture(base_file_path=base_file_path, is_foot_always_up=True)
    
    # Handle special cases to skip certain captures or transitions
    if MOCAP_FP_capture_name == '22_MNTRL_MC_V1' and str(row['tx']) == "3":
        continue
    elif MOCAP_FP_capture_name == '24_MNTRR_MC_V1' and str(row['tx']) == "2":
        continue

    # Calculate average COP velocity per frame
    force_plate_capture.calculate_avg_COP_velocity_per_frame()

    # Check if the dictionary is populated
    if not force_plate_capture.avg_COP_per_frame_dict:
        print(f"No average COP velocity data to plot for capture {force_plate_capture.RADAR_Capture}.")
    else:
        # Plot the average COP velocity per frame for each tx
        for tx, avg_velocities in force_plate_capture.avg_COP_per_frame_dict.items():
            if not avg_velocities:
                print(f"No data to plot for tx {tx} in capture {force_plate_capture.RADAR_Capture}.")
            plt.figure(figsize=(12, 6))
            plt.plot(avg_velocities, marker='o', linestyle='-')
            plt.xlabel('Frame Index')
            plt.ylabel('Average COP Velocity (mm/s)')
            plt.title(f'Average COP Velocity per Frame for {force_plate_capture.RADAR_Capture}, tx {tx}')
            plt.grid(True)
            plt.show()
            break
    

In [ ]:


# Iterate over each row in the DataFrame
for index, row in fp_mocap_comparison_df.iterrows():
    participant = row['RADAR_capture'][:2]
    MOCAP_FP_capture_name = row['RADAR_capture'].replace('_RR_', '_MC_')
    base_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_FP_1and2_MNTR/{participant}/{MOCAP_FP_capture_name}.tsv"
    
    # Instantiate the FPDataCapture object with is_foot_always_up=True
    force_plate_capture = FPDataCapture(base_file_path=base_file_path, is_foot_always_up=True)
    
    # Handle special cases to skip certain captures or transitions
    if MOCAP_FP_capture_name == '22_MNTRL_MC_V1' and str(row['tx']) == "3":
        continue
    elif MOCAP_FP_capture_name == '24_MNTRR_MC_V1' and str(row['tx']) == "2":
        continue
    
    # Get t_stable and t_end
    t_stable = row['t_stable']
    t_end = row['t_foot_down'] if pd.isnull(row['t_break']) else row['t_break']
    
    # Isolate the data between t_stable and t_end
    filtered_data = force_plate_capture.isolate_rows_by_time(t_stable, t_end)
    
    # Check if there are enough unique COP points to compute velocity
    if len(filtered_data.drop_duplicates(subset=['COP_X', 'COP_Y'])) >= 3:
        # Compute the average speed over the time period
        velocity = force_plate_capture.average_speed(filtered_data)
        if velocity is not None:
            # Update the DataFrame directly using the index
            fp_mocap_comparison_df.at[index, 'fp_cop_time_norm_length'] = velocity
            print(f"Velocity for {MOCAP_FP_capture_name}, tx {row['tx']} is: {velocity}")
        else:
            print(f"Total time is zero or negative for {MOCAP_FP_capture_name}, tx {row['tx']}")
    else:
        print(f"Not enough points to calculate velocity for {MOCAP_FP_capture_name}, tx {row['tx']}")


Load Time frames V3 and generate table to be updated 

In [ ]:
event_csv_path = "/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/MOCAP_FP_RADAR_FU_Stable_Break_FD_TIME_FRAMES_v3.csv"

event_labels_df = pd.read_csv(event_csv_path)

fp_mocap_comparison_df = event_labels_df
fp_mocap_comparison_df.drop(columns=['t_foot_up', 'MOCAP_Start_Time', 'RADAR_Start_Frame', 'RADAR_End_Frame', 'MOCAP_End_Time'], inplace=True)

fp_mocap_comparison_df['frame_break'] = fp_mocap_comparison_df['frame_break'].fillna(fp_mocap_comparison_df['frame_end'])

fp_mocap_comparison_df['t_break'] = fp_mocap_comparison_df['t_break'].fillna(fp_mocap_comparison_df['t_foot_down'])

fp_mocap_comparison_df['frame_break'] = fp_mocap_comparison_df['frame_break'].astype(int)
print(fp_mocap_comparison_df.head())

2. Compute Average Marker Speeds and COP Velocities

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Assuming 'fp_mocap_comparison_df' and 'MOCAPDataCapture' are already defined

# Select a sample capture from your DataFrame
sample_index = 0  # Change this index to select a different capture
row = fp_mocap_comparison_df.iloc[sample_index]

# Determine t_start and t_end based on 'is_final_tx'
if row['is_final_tx']:
    t_start = row['t_stable']
    t_end = row['t_foot_down']
else:
    t_start = row['t_stable']
    t_end = row['t_break']

# Check for NaN values
if pd.isna(t_start) or pd.isna(t_end):
    print("Time values are missing.")
    exit()

print(f"Time interval: {t_start} to {t_end} seconds")



In [ ]:
# Get participant and capture names
participant = row['RADAR_capture'][:2]
MOCAP_capture = row['RADAR_capture'].replace('_RR_', '_MC_')
MOCAP_spot_check_path = f"/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/{participant}/tsv/{MOCAP_capture}.tsv"

# Instantiate the MOCAPDataCapture class
mocap = MOCAPDataCapture(MOCAP_spot_check_path)

# Extract MOCAP velocity data between t_start and t_end
mocap_vel_data = mocap.velocity_data
# Filter for time between t_start and t_end
mocap_vel_data_interval = mocap_vel_data[(mocap_vel_data['time'] >= t_start) & (mocap_vel_data['time'] <= t_end)]


In [ ]:

# Extract MOCAP velocity data between t_start and t_end
mocap_vel_data = mocap.velocity_data
mocap_vel_data_interval = mocap_vel_data[(mocap_vel_data['time'] >= t_start) & (mocap_vel_data['time'] <= t_end)]

# Set the base file path for COP data
MOCAP_FP_capture_name = row['RADAR_capture'].replace('_RR_', '_MC_')
base_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_FP_1and2_MNTR/{participant}/{MOCAP_FP_capture_name}.tsv"

# Instantiate FPDataCapture for COP data
force_plate_capture = FPDataCapture(base_file_path)

# Select raw data based on 'MNTRL' or 'MNTRR' in the base file path
if "MNTRR" in force_plate_capture.base_file_path:
    cop_data = force_plate_capture.data_f_1
elif "MNTRL" in force_plate_capture.base_file_path:
    cop_data = force_plate_capture.data_f_2
else:
    print("Unknown base file path format.")
    exit()

# Extract COP data between t_stable and t_break (or t_foot_down if final tx)
cop_data_interval = cop_data[(cop_data['time'] >= t_start) & (cop_data['time'] <= t_end)]

# Calculate COP speed as sqrt(COP_X^2 + COP_Y^2) at 1200 Hz sampling rate
cop_x = cop_data_interval['COP_X'].values
cop_y = cop_data_interval['COP_Y'].values
cop_speed = np.sqrt(cop_x**2 + cop_y**2)

# Extract COP times (sampling rate is 1200 Hz)
cop_time = cop_data_interval['time'].values

# Extract velocities for multiple markers from MOCAP
markers = ['Shoulder_L', 'Shoulder_R', 'Wrist_L', 'Wrist_R', 'Chest', 'Belly']
marker_speeds_dict = {}

for marker in markers:
    vel_columns = [f"{marker}_vel_X", f"{marker}_vel_Y", f"{marker}_vel_Z"]
    if not all(col in mocap_vel_data_interval.columns for col in vel_columns):
        print(f"Velocity data for {marker} is missing.")
        continue
    marker_velocities = mocap_vel_data_interval[vel_columns].values
    speeds = np.linalg.norm(marker_velocities, axis=1)
    marker_speeds_dict[marker] = speeds

# Compute average speeds for bilateral markers
def average_marker_speeds(marker_left, marker_right):
    speeds_left = marker_speeds_dict.get(marker_left)
    speeds_right = marker_speeds_dict.get(marker_right)
    if speeds_left is None or speeds_right is None:
        print(f"Missing data for {marker_left} or {marker_right}.")
        return None
    avg_speeds = (speeds_left + speeds_right) / 2
    return avg_speeds

shoulder_avg_speeds = average_marker_speeds('Shoulder_L', 'Shoulder_R')
wrist_avg_speeds = average_marker_speeds('Wrist_L', 'Wrist_R')

# Update marker_speeds_dict with averaged speeds
if shoulder_avg_speeds is not None:
    marker_speeds_dict['Shoulder_avg'] = shoulder_avg_speeds
if wrist_avg_speeds is not None:
    marker_speeds_dict['Wrist_avg'] = wrist_avg_speeds

# Remove individual sides from the dictionary
for marker in ['Shoulder_L', 'Shoulder_R', 'Wrist_L', 'Wrist_R']:
    if marker in marker_speeds_dict:
        del marker_speeds_dict[marker]

# Plot marker speeds and COP speed over time
plt.figure(figsize=(12, 6))

# Plot marker speeds
times = mocap_vel_data_interval['time'].values
for marker, speeds in marker_speeds_dict.items():
    plt.plot(times, speeds, label=f'{marker} Speed')

# Plot COP speed
plt.plot(cop_time, cop_speed, label='COP Speed', linestyle='--', color='black')

plt.xlabel('Time (s)')
plt.ylabel('Speed (units/s)')
plt.title(f'COP Speed and Marker Velocities for Capture: {MOCAP_capture}')
plt.legend()
plt.grid(True)
plt.show()


loop code, to check many FP vs MOCAP and calculate correlation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import pearsonr

# Assuming FPDataCapture and MOCAPDataCapture classes are available
# from FPDataCapture import FPDataCapture
# from MOCAPDataCapture import MOCAPDataCapture

for idx, row in fp_mocap_comparison_df.iterrows():   
    # Determine t_start and t_end based on 'is_final_tx'
    if row['is_final_tx']:
        t_start = row['t_stable']
        t_end = row['t_foot_down']
    else:
        t_start = row['t_stable']
        t_end = row['t_break']

    # Check for NaN values
    if pd.isna(t_start) or pd.isna(t_end):
        print(f"Time values are missing for sample {idx}.")
        continue

    # Get participant and capture names
    participant = row['RADAR_capture'][:2]
    
    if participant != "15" and participant != "24":
        continue
    
    MOCAP_capture = row['RADAR_capture'].replace('_RR_', '_MC_')
    MOCAP_spot_check_path = f"/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/{participant}/tsv/{MOCAP_capture}.tsv"

    # Instantiate the MOCAPDataCapture class
    mocap = MOCAPDataCapture(MOCAP_spot_check_path)

    # Extract MOCAP velocity data between t_start and t_end
    mocap_vel_data = mocap.velocity_data
    mocap_vel_data_interval = mocap_vel_data[(mocap_vel_data['time'] >= t_start) & (mocap_vel_data['time'] <= t_end)]

    # Set the base file path for COP data
    MOCAP_FP_capture_name = row['RADAR_capture'].replace('_RR_', '_MC_')
    base_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_FP_1and2_MNTR/{participant}/{MOCAP_FP_capture_name}.tsv"

    # Instantiate FPDataCapture for COP data
    force_plate_capture = FPDataCapture(base_file_path)

    # Select raw data based on 'MNTRL' or 'MNTRR' in the base file path
    if "MNTRR" in force_plate_capture.base_file_path:
        cop_data = force_plate_capture.data_f_1
    elif "MNTRL" in force_plate_capture.base_file_path:
        cop_data = force_plate_capture.data_f_2
    else:
        print(f"Unknown base file path format for sample {sample_index}.")
        continue

    # Extract COP data between t_stable and t_break (or t_foot_down if final tx)
    cop_data_interval = cop_data[(cop_data['time'] >= t_start) & (cop_data['time'] <= t_end)]

    # Calculate COP speed as sqrt(COP_X^2 + COP_Y^2) at 1200 Hz sampling rate
    cop_x = cop_data_interval['COP_X'].values
    cop_y = cop_data_interval['COP_Y'].values
    cop_speed = np.sqrt(cop_x**2 + cop_y**2)

    # Extract COP times
    cop_time = cop_data_interval['time'].values

    # Extract velocities for multiple markers from MOCAP
    markers = ['Shoulder_L', 'Shoulder_R', 'Wrist_L', 'Wrist_R', 'Chest', 'Belly']
    marker_speeds_dict = {}

    for marker in markers:
        vel_columns = [f"{marker}_vel_X", f"{marker}_vel_Y", f"{marker}_vel_Z"]
        if not all(col in mocap_vel_data_interval.columns for col in vel_columns):
            print(f"Velocity data for {marker} is missing.")
            continue
        marker_velocities = mocap_vel_data_interval[vel_columns].values
        speeds = np.linalg.norm(marker_velocities, axis=1)
        marker_speeds_dict[marker] = speeds

    # Compute average speeds for bilateral markers
    def average_marker_speeds(marker_left, marker_right):
        speeds_left = marker_speeds_dict.get(marker_left)
        speeds_right = marker_speeds_dict.get(marker_right)
        if speeds_left is None or speeds_right is None:
            print(f"Missing data for {marker_left} or {marker_right}.")
            return None
        avg_speeds = (speeds_left + speeds_right) / 2
        return avg_speeds

    shoulder_avg_speeds = average_marker_speeds('Shoulder_L', 'Shoulder_R')
    wrist_avg_speeds = average_marker_speeds('Wrist_L', 'Wrist_R')

    # Update marker_speeds_dict with averaged speeds
    if shoulder_avg_speeds is not None:
        marker_speeds_dict['Shoulder_avg'] = shoulder_avg_speeds
    if wrist_avg_speeds is not None:
        marker_speeds_dict['Wrist_avg'] = wrist_avg_speeds

    # Remove individual sides from the dictionary
    for marker in ['Shoulder_L', 'Shoulder_R', 'Wrist_L', 'Wrist_R']:
        if marker in marker_speeds_dict:
            del marker_speeds_dict[marker]

    # Plot marker speeds and COP speed over time
    plt.figure(figsize=(12, 6))

    # Plot marker speeds
    times = mocap_vel_data_interval['time'].values
    for marker, speeds in marker_speeds_dict.items():
        plt.plot(times, speeds, label=f'{marker} Speed')

    # Plot COP speed
    plt.plot(cop_time, cop_speed, label='COP Speed', linestyle='--', color='black')

    plt.xlabel('Time (s)')
    plt.ylabel('Speed (units/s)')
    plt.title(f'COP Speed and Marker Velocities for Capture: {MOCAP_capture}')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Calculate correlations between COP speed and marker speeds
    for marker, speeds in marker_speeds_dict.items():
        min_length = min(len(cop_speed), len(speeds))
        cop_speed_resampled = cop_speed[:min_length]
        speeds_resampled = speeds[:min_length]

        correlation, _ = pearsonr(cop_speed_resampled, speeds_resampled)
        print(f"Correlation between COP Speed and {marker} Speed: {correlation:.4f}")


MOCAP to FP correlations

In [ ]:
import numpy as np
from scipy.stats import pearsonr
import os

# Initialize empty lists to store correlation results
chest_correlations = []
belly_correlations = []
shoulder_avg_correlations = []
wrist_avg_correlations = []

# Iterate over each row in the comparison DataFrame
for index, row in fp_mocap_comparison_df.iterrows():
    

    
    print(row['tx'])
    
    # Determine t_start and t_end based on 'is_final_tx'
    if row['is_final_tx']:
        t_start = row['t_stable']
        t_end = row['t_foot_down']
    else:
        t_start = row['t_stable']
        t_end = row['t_break']

    # Check for NaN values
    if pd.isna(t_start) or pd.isna(t_end):
        print(f"Skipping sample {index} due to missing time values.")
        continue

    # Get participant and capture names
    participant = row['RADAR_capture'][:2]
    MOCAP_capture = row['RADAR_capture'].replace('_RR_', '_MC_')
    MOCAP_spot_check_path = f"/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/{participant}/tsv/{MOCAP_capture}.tsv"

    # Instantiate the MOCAPDataCapture class
    mocap = MOCAPDataCapture(MOCAP_spot_check_path)

    # Extract MOCAP velocity data between t_start and t_end
    mocap_vel_data = mocap.velocity_data
    mocap_vel_data_interval = mocap_vel_data[(mocap_vel_data['time'] >= t_start) & (mocap_vel_data['time'] <= t_end)]

    # Set the base file path for COP data
    MOCAP_FP_capture_name = row['RADAR_capture'].replace('_RR_', '_MC_')
    base_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_FP_1and2_MNTR/{participant}/{MOCAP_FP_capture_name}.tsv"

    # Instantiate FPDataCapture for COP data
    force_plate_capture = FPDataCapture(base_file_path)
    
    # Handle special cases to skip certain captures or transitions
    if MOCAP_FP_capture_name == '22_MNTRL_MC_V1' and str(row['tx']) == "3":
        continue
    elif MOCAP_FP_capture_name == '24_MNTRR_MC_V1' and str(row['tx']) == "2":
        continue
    elif MOCAP_FP_capture_name == '12_MNTRR_MC_V1':
        continue

    # Select raw data based on 'MNTRL' or 'MNTRR' in the base file path
    if "MNTRR" in force_plate_capture.base_file_path:
        cop_data = force_plate_capture.data_f_1
    elif "MNTRL" in force_plate_capture.base_file_path:
        cop_data = force_plate_capture.data_f_2
    else:
        print(f"Unknown base file path format for sample {index}.")
        continue

    # Extract COP data between t_stable and t_break (or t_foot_down if final tx)
    cop_data_interval = cop_data[(cop_data['time'] >= t_start) & (cop_data['time'] <= t_end)]

    # Calculate COP speed as sqrt(COP_X^2 + COP_Y^2) at 1200 Hz sampling rate
    cop_x = cop_data_interval['COP_X'].values
    cop_y = cop_data_interval['COP_Y'].values
    cop_speed = np.sqrt(cop_x**2 + cop_y**2)

    # Extract COP times
    cop_time = cop_data_interval['time'].values

    # Extract velocities for multiple markers from MOCAP
    markers = ['Shoulder_L', 'Shoulder_R', 'Wrist_L', 'Wrist_R', 'Chest', 'Belly']
    marker_speeds_dict = {}

    for marker in markers:
        vel_columns = [f"{marker}_vel_X", f"{marker}_vel_Y", f"{marker}_vel_Z"]
        if not all(col in mocap_vel_data_interval.columns for col in vel_columns):
            print(f"Velocity data for {marker} is missing.")
            continue
        marker_velocities = mocap_vel_data_interval[vel_columns].values
        speeds = np.linalg.norm(marker_velocities, axis=1)
        marker_speeds_dict[marker] = speeds

    # Compute average speeds for bilateral markers
    def average_marker_speeds(marker_left, marker_right):
        speeds_left = marker_speeds_dict.get(marker_left)
        speeds_right = marker_speeds_dict.get(marker_right)
        if speeds_left is None or speeds_right is None:
            print(f"Missing data for {marker_left} or {marker_right}.")
            return None
        avg_speeds = (speeds_left + speeds_right) / 2
        return avg_speeds

    shoulder_avg_speeds = average_marker_speeds('Shoulder_L', 'Shoulder_R')
    wrist_avg_speeds = average_marker_speeds('Wrist_L', 'Wrist_R')

    # Update marker_speeds_dict with averaged speeds
    if shoulder_avg_speeds is not None:
        marker_speeds_dict['Shoulder_avg'] = shoulder_avg_speeds
    if wrist_avg_speeds is not None:
        marker_speeds_dict['Wrist_avg'] = wrist_avg_speeds

    # Remove individual sides from the dictionary
    for marker in ['Shoulder_L', 'Shoulder_R', 'Wrist_L', 'Wrist_R']:
        if marker in marker_speeds_dict:
            del marker_speeds_dict[marker]

    # Calculate correlations between COP speed and marker speeds
    for marker, speeds in marker_speeds_dict.items():
        min_length = min(len(cop_speed), len(speeds))
        cop_speed_resampled = cop_speed[:min_length]
        speeds_resampled = speeds[:min_length]

        if marker == 'Chest':
            correlation, _ = pearsonr(cop_speed_resampled, speeds_resampled)
            chest_correlations.append(correlation)
        elif marker == 'Belly':
            correlation, _ = pearsonr(cop_speed_resampled, speeds_resampled)
            belly_correlations.append(correlation)
        elif marker == 'Shoulder_avg':
            correlation, _ = pearsonr(cop_speed_resampled, speeds_resampled)
            shoulder_avg_correlations.append(correlation)
        elif marker == 'Wrist_avg':
            correlation, _ = pearsonr(cop_speed_resampled, speeds_resampled)
            wrist_avg_correlations.append(correlation)

# Summarize the results
print(f"Average Correlation between COP Speed and Chest Speed: {np.mean(chest_correlations):.4f}")
print(f"Average Correlation between COP Speed and Belly Speed: {np.mean(belly_correlations):.4f}")
print(f"Average Correlation between COP Speed and Shoulder_avg Speed: {np.mean(shoulder_avg_correlations):.4f}")
print(f"Average Correlation between COP Speed and Wrist_avg Speed: {np.mean(wrist_avg_correlations):.4f}")


Spearman Rank Correlation

In [5]:
import numpy as np
from scipy.stats import spearmanr
import os

# Initialize empty lists to store correlation results
chest_correlations = []
belly_correlations = []
shoulder_avg_correlations = []
wrist_avg_correlations = []
chest_mad = []
belly_mad = []
shoulder_avg_mad = []
wrist_avg_mad = []

# Iterate over each row in the comparison DataFrame
for index, row in fp_mocap_comparison_df.iterrows():
    
    print(row['tx'])
    
    # Determine t_start and t_end based on 'is_final_tx'
    if row['is_final_tx']:
        t_start = row['t_stable']
        t_end = row['t_foot_down']
    else:
        t_start = row['t_stable']
        t_end = row['t_break']

    # Check for NaN values
    if pd.isna(t_start) or pd.isna(t_end):
        print(f"Skipping sample {index} due to missing time values.")
        continue

    # Get participant and capture names
    participant = row['RADAR_capture'][:2]
    MOCAP_capture = row['RADAR_capture'].replace('_RR_', '_MC_')
    MOCAP_spot_check_path = f"/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/{participant}/tsv/{MOCAP_capture}.tsv"

    # Instantiate the MOCAPDataCapture class
    mocap = MOCAPDataCapture(MOCAP_spot_check_path)

    # Extract MOCAP velocity data between t_start and t_end
    mocap_vel_data = mocap.velocity_data
    mocap_vel_data_interval = mocap_vel_data[(mocap_vel_data['time'] >= t_start) & (mocap_vel_data['time'] <= t_end)]

    # Set the base file path for COP data
    MOCAP_FP_capture_name = row['RADAR_capture'].replace('_RR_', '_MC_')
    base_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_FP_1and2_MNTR/{participant}/{MOCAP_FP_capture_name}.tsv"

    # Instantiate FPDataCapture for COP data
    force_plate_capture = FPDataCapture(base_file_path)
    
    # Handle special cases to skip certain captures or transitions
    if MOCAP_FP_capture_name == '22_MNTRL_MC_V1' and str(row['tx']) == "3":
        continue
    elif MOCAP_FP_capture_name == '24_MNTRR_MC_V1' and str(row['tx']) == "2":
        continue
    elif MOCAP_FP_capture_name == '12_MNTRR_MC_V1':
        continue

    # Select raw data based on 'MNTRL' or 'MNTRR' in the base file path
    if "MNTRR" in force_plate_capture.base_file_path:
        cop_data = force_plate_capture.data_f_1
    elif "MNTRL" in force_plate_capture.base_file_path:
        cop_data = force_plate_capture.data_f_2
    else:
        print(f"Unknown base file path format for sample {index}.")
        continue

    # Extract COP data between t_stable and t_break (or t_foot_down if final tx)
    cop_data_interval = cop_data[(cop_data['time'] >= t_start) & (cop_data['time'] <= t_end)]

    # Calculate COP speed as sqrt(COP_X^2 + COP_Y^2) at 1200 Hz sampling rate
    cop_x = cop_data_interval['COP_X'].values
    cop_y = cop_data_interval['COP_Y'].values
    cop_speed = np.sqrt(cop_x**2 + cop_y**2)

    # Extract velocities for multiple markers from MOCAP
    markers = ['Shoulder_L', 'Shoulder_R', 'Wrist_L', 'Wrist_R', 'Chest', 'Belly']
    marker_speeds_dict = {}

    for marker in markers:
        vel_columns = [f"{marker}_vel_X", f"{marker}_vel_Y", f"{marker}_vel_Z"]
        if not all(col in mocap_vel_data_interval.columns for col in vel_columns):
            print(f"Velocity data for {marker} is missing.")
            continue
        marker_velocities = mocap_vel_data_interval[vel_columns].values
        speeds = np.linalg.norm(marker_velocities, axis=1)
        marker_speeds_dict[marker] = speeds

    # Compute average speeds for bilateral markers
    def average_marker_speeds(marker_left, marker_right):
        speeds_left = marker_speeds_dict.get(marker_left)
        speeds_right = marker_speeds_dict.get(marker_right)
        if speeds_left is None or speeds_right is None:
            print(f"Missing data for {marker_left} or {marker_right}.")
            return None
        avg_speeds = (speeds_left + speeds_right) / 2
        return avg_speeds

    shoulder_avg_speeds = average_marker_speeds('Shoulder_L', 'Shoulder_R')
    wrist_avg_speeds = average_marker_speeds('Wrist_L', 'Wrist_R')

    # Update marker_speeds_dict with averaged speeds
    if shoulder_avg_speeds is not None:
        marker_speeds_dict['Shoulder_avg'] = shoulder_avg_speeds
    if wrist_avg_speeds is not None:
        marker_speeds_dict['Wrist_avg'] = wrist_avg_speeds

    # Remove individual sides from the dictionary
    for marker in ['Shoulder_L', 'Shoulder_R', 'Wrist_L', 'Wrist_R']:
        if marker in marker_speeds_dict:
            del marker_speeds_dict[marker]

    # Calculate absolute value correlations or MAD between COP speed and marker speeds
    for marker, speeds in marker_speeds_dict.items():
        min_length = min(len(cop_speed), len(speeds))
        cop_speed_resampled = cop_speed[:min_length]
        speeds_resampled = speeds[:min_length]

        # Compute Spearman Rank Correlation
        if marker == 'Chest':
            correlation, _ = spearmanr(cop_speed_resampled, speeds_resampled)
            chest_correlations.append(correlation)
            chest_mad.append(np.mean(np.abs(cop_speed_resampled - speeds_resampled)))
        elif marker == 'Belly':
            correlation, _ = spearmanr(cop_speed_resampled, speeds_resampled)
            belly_correlations.append(correlation)
            belly_mad.append(np.mean(np.abs(cop_speed_resampled - speeds_resampled)))
        elif marker == 'Shoulder_avg':
            correlation, _ = spearmanr(cop_speed_resampled, speeds_resampled)
            shoulder_avg_correlations.append(correlation)
            shoulder_avg_mad.append(np.mean(np.abs(cop_speed_resampled - speeds_resampled)))
        elif marker == 'Wrist_avg':
            correlation, _ = spearmanr(cop_speed_resampled, speeds_resampled)
            wrist_avg_correlations.append(correlation)
            wrist_avg_mad.append(np.mean(np.abs(cop_speed_resampled - speeds_resampled)))

# Summarize the results
print(f"Average Spearman Correlation between COP Speed and Chest Speed: {np.mean(chest_correlations):.4f}")
print(f"Average Mean Absolute Difference (MAD) between COP Speed and Chest Speed: {np.mean(chest_mad):.4f}")

print(f"Average Spearman Correlation between COP Speed and Belly Speed: {np.mean(belly_correlations):.4f}")
print(f"Average MAD between COP Speed and Belly Speed: {np.mean(belly_mad):.4f}")

print(f"Average Spearman Correlation between COP Speed and Shoulder_avg Speed: {np.mean(shoulder_avg_correlations):.4f}")
print(f"Average MAD between COP Speed and Shoulder_avg Speed: {np.mean(shoulder_avg_mad):.4f}")

print(f"Average Spearman Correlation between COP Speed and Wrist_avg Speed: {np.mean(wrist_avg_correlations):.4f}")
print(f"Average MAD between COP Speed and Wrist_avg Speed: {np.mean(wrist_avg_mad):.4f}")


1
Processing File: 01_MNTRL_MC_V1.tsv
/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/01/tsv/01_MNTRL_MC_V1_pos.tsv
/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/01/tsv/01_MNTRL_MC_V1_vel.tsv
2
Processing File: 01_MNTRL_MC_V1.tsv
/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/01/tsv/01_MNTRL_MC_V1_pos.tsv
/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/01/tsv/01_MNTRL_MC_V1_vel.tsv
3
Processing File: 01_MNTRL_MC_V1.tsv
/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/01/tsv/01_MNTRL_MC_V1_pos.tsv
/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/01/tsv/01_MNTRL_MC_V1_vel.tsv
1
Processing File: 01_MNTRL_MC_V2.tsv
/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/01/tsv/01_MNTRL_MC_V2_pos.tsv
/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/01/tsv/01_MNTRL_MC_V2_vel.tsv
2
Processing File: 01_MNTRL_MC_V2.tsv
/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/01/tsv/01_MNTRL_MC_V2_pos.tsv
/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/01/tsv/01_MNTRL_MC_V2_vel.tsv
3
Processing File: 01_MNTRL_MC_V2.tsv
/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data

Add Time-Normalized Force Plate data to new dataframe

In [ ]:
# Ensure the DataFrame has the new column initialized to NaN or some default value
fp_mocap_comparison_df['fp_cop_time_norm_length'] = np.nan

for index, row in fp_mocap_comparison_df.iterrows():
    participant = row['RADAR_capture'][:2]
    MOCAP_FP_capture_name = row['RADAR_capture'].replace('_RR_', '_MC_')
    base_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_FP_1and2_MNTR/{participant}/{MOCAP_FP_capture_name}.tsv"
    force_plate_capture = FPDataCapture(base_file_path=base_file_path, is_foot_always_up=True)
    
    t_stable = row['t_stable']
    t_end = row['t_foot_down'] if np.isnan(row['t_break']) else row['t_break']
    
    if MOCAP_FP_capture_name == '22_MNTRL_MC_V1' and row['tx'] == "3":
        continue
    elif MOCAP_FP_capture_name == '24_MNTRR_MC_V1' and row['tx'] == "2":
        continue
    
    filtered_data = force_plate_capture.isolate_rows_by_time(t_stable, t_end)
    velocity = None  # Default value in case of insufficient data points
    
    if len(filtered_data.drop_duplicates(subset=['COP_X', 'COP_Y'])) >= 3:
        velocity = force_plate_capture.average_speed(filtered_data)
        # Update the DataFrame directly using the index
        fp_mocap_comparison_df.at[index, 'fp_cop_time_norm_length'] = velocity
        print(f"velocity for {MOCAP_FP_capture_name}, tx {row['tx']} is: {velocity}")
    else:
        print(f"Not enough points to calculate velocity for {MOCAP_FP_capture_name}, tx {row['tx']}")
        

    

R vs L FP analysis. Do they Differ? 

In [ ]:
import pandas as pd
from scipy.stats import ttest_rel, ttest_ind

# Assuming fp_mocap_comparison_df is your pandas DataFrame
# Extract individual ID and capture side
fp_mocap_comparison_df['individual'] = fp_mocap_comparison_df['RADAR_capture'].str[:2]
fp_mocap_comparison_df['side'] = fp_mocap_comparison_df['RADAR_capture'].str[7]

# Split the dataframe into two based on the capture side
fp_mocap_comparison_df_R =fp_mocap_comparison_df[fp_mocap_comparison_df['side'] == 'R']
fp_mocap_comparison_df_L = fp_mocap_comparison_df[fp_mocap_comparison_df['side'] == 'L']

# Ensure we have matching captures for each individual
individuals = set(fp_mocap_comparison_df_R['individual']).intersection(set(fp_mocap_comparison_df_L['individual']))

# Initialize a list to store the results
results = []

for individual in individuals:
    # Extract the 'fp_cop_time_norm_length' for R and L captures of the current individual
    data_R = fp_mocap_comparison_df_R[fp_mocap_comparison_df_R['individual'] == individual]['fp_cop_time_norm_length']
    data_L = fp_mocap_comparison_df_L[fp_mocap_comparison_df_L['individual'] == individual]['fp_cop_time_norm_length']
    
    
    # Calculate the means
    mean_R = data_R.mean()
    mean_L = data_L.mean()

    # Perform the independent t-test
    t_stat, p_val = ttest_ind(data_R, data_L, equal_var=False)  # Welch's t-test
    results.append((individual, len(data_R), mean_R, len(data_L), mean_L, p_val))

# Convert the results to a DataFrame for nicer display
results_df = pd.DataFrame(results, columns=['Individual', 'Count_R', 'Mean_R', 'Count_L', 'Mean_L', 'P-Value'])

print(results_df)


In [ ]:
print(fp_mocap_comparison_df.head())

Load MOCAP Data

In [ ]:
# Ensure the necessary columns exist in the DataFrame
for marker in ['Shoulder', 'Wrist', 'Chest', 'Belly']:
    if marker in ['Shoulder', 'Wrist']:
        fp_mocap_comparison_df[f'{marker}_avg_speed'] = np.nan
    else:
        fp_mocap_comparison_df[f'{marker}_speed'] = np.nan

for index, row in fp_mocap_comparison_df.iterrows():
    participant = row['RADAR_capture'][:2]
    MOCAP_capture = row['RADAR_capture'].replace('_RR_', '_MC_')
    MOCAP_spot_check_path = f"/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/{participant}/tsv/{MOCAP_capture}.tsv"
    
    mocap = MOCAPDataCapture(MOCAP_spot_check_path)
    time_normalized_lengths = mocap.get_time_normalized_length(row['t_stable'], row['t_break'], ['Shoulder', 'Wrist', 'Chest', 'Belly'])
    
    # Update the DataFrame with calculated speeds
    for marker in time_normalized_lengths:
        if marker in ['Shoulder_R', 'Shoulder_L', 'Wrist_R', 'Wrist_L']:
            # Extract base marker name and side
            base_marker, side = marker.rsplit('_', 1)
            # Average the R and L values and update the corresponding column
            if f'{base_marker}_R' in time_normalized_lengths and f'{base_marker}_L' in time_normalized_lengths:
                avg_speed = np.mean([time_normalized_lengths[f'{base_marker}_R'], time_normalized_lengths[f'{base_marker}_L']])
                fp_mocap_comparison_df.at[index, f'{base_marker}_avg_speed'] = avg_speed
        else:
            # Directly update the column for non-side-specific markers
            fp_mocap_comparison_df.at[index, f'{marker}_speed'] = time_normalized_lengths[marker]
    
    print(f"Updated row {index} with time normalized lengths")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Assuming fp_mocap_comparison_df and MOCAPDataCapture are already defined

# Select a sample capture from your DataFrame
sample_index = 0  # Change this index to select a different capture
row = fp_mocap_comparison_df.iloc[sample_index]

participant = row['RADAR_capture'][:2]
MOCAP_capture = row['RADAR_capture'].replace('_RR_', '_MC_')
MOCAP_spot_check_path = f"/Volumes/FourTBLaCie/Yoga_Study_MOCAP_Data/{participant}/tsv/{MOCAP_capture}.tsv"

# Instantiate the MOCAPDataCapture class
mocap = MOCAPDataCapture(MOCAP_spot_check_path)


In [ ]:
# Define the markers to extract
markers = ['Shoulder_R', 'Shoulder_L', 'Wrist_R', 'Wrist_L', 'Chest', 'Belly']

# Initialize dictionaries to store velocities
velocities = {}
times = mocap.velocity_data['time'].values  # Time array

# Extract velocities for each marker
for marker in markers:
    # Velocity columns for the marker
    vel_columns = [f"{marker}_vel_X", f"{marker}_vel_Y", f"{marker}_vel_Z"]
    
    # Check if the columns exist
    if not all(col in mocap.velocity_data.columns for col in vel_columns):
        print(f"Velocity data for {marker} is missing.")
        continue
    
    # Extract the velocities
    marker_velocities = mocap.velocity_data[vel_columns].values  # Shape: (num_frames, 3)
    
    # Compute speed (magnitude of velocity)
    marker_speeds = np.linalg.norm(marker_velocities, axis=1)
    
    velocities[marker] = marker_speeds


In [ ]:
# Function to compute average speeds for bilateral markers
def average_marker_speeds(marker_left, marker_right):
    speeds_left = velocities.get(marker_left)
    speeds_right = velocities.get(marker_right)
    if speeds_left is None or speeds_right is None:
        print(f"Missing data for {marker_left} or {marker_right}.")
        return None
    avg_speeds = (speeds_left + speeds_right) / 2
    return avg_speeds

# Compute average speeds for 'Shoulder' and 'Wrist'
shoulder_avg_speeds = average_marker_speeds('Shoulder_L', 'Shoulder_R')
wrist_avg_speeds = average_marker_speeds('Wrist_L', 'Wrist_R')


In [ ]:
# Plot the speeds
plt.figure(figsize=(12, 6))

# Plot Shoulder average speed
if shoulder_avg_speeds is not None:
    plt.plot(times, shoulder_avg_speeds, label='Shoulder Average Speed')
else:
    print("Shoulder average speeds not available.")

# Plot Wrist average speed
if wrist_avg_speeds is not None:
    plt.plot(times, wrist_avg_speeds, label='Wrist Average Speed')
else:
    print("Wrist average speeds not available.")

# Plot Chest speed
if 'Chest' in velocities:
    plt.plot(times, velocities['Chest'], label='Chest Speed')
else:
    print("Chest speed data not available.")

# Plot Belly speed
if 'Belly' in velocities:
    plt.plot(times, velocities['Belly'], label='Belly Speed')
else:
    print("Belly speed data not available.")

# Customize the plot
plt.xlabel('Time (s)')
plt.ylabel('Speed (units/s)')
plt.title(f'Velocity vs. Time for Capture: {MOCAP_capture}')
plt.legend()
plt.grid(True)
plt.show()


Add Time-Normalized MOCAP Data to new dataframe

Calculate Time-Norm for various markers

Add to dataframe

Run Statistical Analysis and Plot Results

In [ ]:
## Extract the first two digits of the RADAR_capture as participant_id
fp_mocap_comparison_df['participant_id'] = fp_mocap_comparison_df['RADAR_capture'].str[:2]

# Creating the plot
plt.figure(figsize=(15, 8))  # Adjust the size as needed to resemble the provided plot
sns.swarmplot(x='participant_id', y='Wrist_avg_speed', data=fp_mocap_comparison_df, palette="colorblind")

# Setting title and labels to match the provided plot
plt.title('Average Wrist Time Normalized Length by Participant')
plt.xlabel('Participant')
plt.ylabel('Average Time Normalized Length (mm/second)')

# This will save the plot to a file on your local system
plt.savefig('Wrist_avg_speed_by_participant.png')

plt.show()

In [ ]:
## Extract the first two digits of the RADAR_capture as participant_id
fp_mocap_comparison_df['participant_id'] = fp_mocap_comparison_df['RADAR_capture'].str[:2]

# Creating the plot
plt.figure(figsize=(15, 8))  # Adjust the size as needed to resemble the provided plot
sns.swarmplot(x='participant_id', y='Chest_speed', data=fp_mocap_comparison_df, palette="colorblind")

# Setting title and labels to match the provided plot
plt.title('Average Chest Time Normalized Length by Participant')
plt.xlabel('Participant')
plt.ylabel('Average Chest Time Normalized Length (mm/second)')

# This will save the plot to a file on your local system
plt.savefig('chest_speed_by_participant.png')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import mplcursors

# Extract the first two digits of the RADAR_capture as participant_id
fp_mocap_comparison_df['participant_id'] = fp_mocap_comparison_df['RADAR_capture'].str[:2]

# Create the plot
plt.figure(figsize=(15, 8))
sns.set(style="whitegrid")  # Set style for seaborn plots
plot = sns.scatterplot(x='participant_id', y='Chest_speed', data=fp_mocap_comparison_df, hue='participant_id')

# Add mplcursors for interactivity with hover
mplcursors.cursor(hover=True).connect(
    "add", lambda sel: sel.annotation.set_text(
        f"RADAR: {fp_mocap_comparison_df.iloc[sel.target.index]['RADAR_capture']}\n"
        f"tx: {fp_mocap_comparison_df.iloc[sel.target.index]['tx']}"
    )
)

plt.title('Chest Speed by Participant')
plt.xlabel('Participant ID')
plt.ylabel('Chest Speed')
plt.show()

Highest Chest speeds

In [ ]:
# Now sort the DataFrame by 'Chest_speed' in descending order and take the first 5 rows
top_chest_speeds = fp_mocap_comparison_df.nlargest(5, 'Chest_speed')

# Print the top 5 rows with the highest 'Chest_speed'
top_chest_speeds

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'fp_mocap_comparison_df' is your DataFrame and it includes the following columns
# For demonstration, I'm creating a mock DataFrame with random data
np.random.seed(0)  # For reproducibility

temp_matrix = fp_mocap_comparison_df[['fp_cop_time_norm_length',	'Shoulder_avg_speed','Wrist_avg_speed',	'Chest_speed',	'Belly_speed']]

# Calculating the correlation matrix
correlation_matrix = temp_matrix.corr()

# Creating the heatmap for the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', square=True, linewidths=.5)
plt.title('Correlation Matrix for All Variables')
plt.show()